In [ ]:
!wget 'https://raw.githubusercontent.com/deepset-ai/COVID-QA/master/data/question-answering/COVID-QA.json'

--2021-04-14 02:25:28--  https://raw.githubusercontent.com/deepset-ai/COVID-QA/master/data/question-answering/COVID-QA.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4418117 (4.2M) [text/plain]
Saving to: ‘COVID-QA.json’

COVID-QA.json       100%[===================>]   4.21M  --.-KB/s    in 0.09s   

2021-04-14 02:25:29 (47.4 MB/s) - ‘COVID-QA.json’ saved [4418117/4418117]



In [ ]:
raw_file = './COVID-QA.json'

In [ ]:
import pandas as pd
import numpy as np
data = pd.read_json(raw_file)
data = pd.json_normalize(data['data'], record_path='paragraphs')
new_data = pd.DataFrame(0, index=np.arange(2019),columns=["document_id", "context", "id_question", "question", "answers_start", "answers_text", "is_impossible"])
idx = 0
for x in range(0,len(data['qas'])):
  for y in range(0, len(data['qas'][x])):
    new_data['document_id'].iloc[idx]=data['document_id'][x]
    new_data['context'].iloc[idx]=data['context'][x]
    new_data['id_question'].iloc[idx] = data['qas'][x][y]['id']
    new_data['question'].iloc[idx] = data['qas'][x][y]['question']
    new_data['answers_start'].iloc[idx] = data['qas'][x][y]['answers'][0]['answer_start']
    new_data['answers_text'].iloc[idx] = data['qas'][x][y]['answers'][0]['text']
    new_data['is_impossible'].iloc[idx] = data['qas'][x][y]['is_impossible']
    idx+=1
new_data


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,document_id,context,id_question,question,answers_start,answers_text,is_impossible
0,630,Functional Genetic Variants in DC-SIGNR Are As...,262,What is the main cause of HIV-1 infection in c...,370,Mother-to-child transmission (MTCT) is the mai...,False
1,630,Functional Genetic Variants in DC-SIGNR Are As...,276,What plays the crucial role in the Mother to C...,2003,DC-SIGNR plays a crucial role in MTCT of HIV-1...,False
2,630,Functional Genetic Variants in DC-SIGNR Are As...,278,How many children were infected by HIV-1 in 20...,2291,"more than 400,000 children were infected world...",False
3,630,Functional Genetic Variants in DC-SIGNR Are As...,316,What is the role of C-C Motif Chemokine Ligand...,28143,"High copy numbers of CCL3L1, a potent HIV-1 su...",False
4,630,Functional Genetic Variants in DC-SIGNR Are As...,305,What is DC-GENR and where is it expressed?,3207,Dendritic cell-specific ICAM-grabbing non-inte...,False
...,...,...,...,...,...,...,...
2014,1713,"Ebola Virus Maintenance: If Not (Only) Bats, W...",5315,What is the structure of the Ebolavirus?,2270,single-strand RNA filoviruses,False
2015,1713,"Ebola Virus Maintenance: If Not (Only) Bats, W...",5316,When was the West African Ebolavirus outbreak?,2546,2013-2016,False
2016,1713,"Ebola Virus Maintenance: If Not (Only) Bats, W...",5317,What animals are considered to be maintenance ...,4083,African bats,False
2017,1713,"Ebola Virus Maintenance: If Not (Only) Bats, W...",5318,What do circles indicate in Figure 1?,7212,a maintenance function play by the host(s),False


## Split training testing

In [ ]:
import pandas as pd 

# Shuffle your dataset 
shuffle_df = new_data.sample(frac=1)

# Define a size for your train set 
train_size = int(0.7 * len(new_data))

# Split your dataset 
train_set = shuffle_df[:train_size]
test_set = shuffle_df[train_size:]

## Mengambi kontext docs dan question

In [ ]:
#ambil context dan question
docs_train = train_set[['document_id','context']]
question_train = train_set[['document_id','id_question','question']]

#ambil context dan question
docs_test = test_set[['document_id','context']]
question_test = test_set[['document_id','id_question','question']]

In [ ]:
docs_train.drop_duplicates(subset=None, keep='first', inplace=True)
docs_test.drop_duplicates(subset=None, keep='first', inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,document_id,context
1509,2555,Backcalculating the Incidence of Infection wit...
276,1548,The First Detection of Equine Coronavirus in A...
689,1660,Hantaviruses in the Americas and Their Role as...
2001,2643,Responding to the COVID-19 pandemic in complex...
1881,2684,\n\n1918 Influenza: the Mother of All Pandemi...
...,...,...
1437,2527,On the Coronavirus (COVID-19) Outbreak and the...
541,2458,Statistics-Based Predictions of Coronavirus Ep...
1745,2653,Molecular and serological investigation of 201...
1478,1722,Recent Progress in Studies of Arterivirus- and...


## Clean the text

In [ ]:
#package
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

### Document

In [ ]:
def clean_data(text):
  lem = WordNetLemmatizer()
  lower_case = text.lower()
  tokenized = word_tokenize(lower_case)
  data_lemmatize=[lem.lemmatize(word,"v") for word in tokenized if not word in stopwords.words('english')]
  data_fixed = ' '.join(data_lemmatize)
  return data_fixed

In [ ]:
docs_clean_train = [clean_data(sentence) for sentence in docs_train['context']]
docs_clean_test = [clean_data(sentence) for sentence in docs_test['context']]

In [ ]:
docs_train['cleaned_data'] = docs_clean_train
docs_test['cleaned_data'] = docs_clean_test

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### Question

In [ ]:
def clean_data_question(text):
  lem = WordNetLemmatizer()
  lower_case = text.lower()
  tokenized = word_tokenize(lower_case)
  data_lemmatize=[lem.lemmatize(word,"v") for word in tokenized]
  data_fixed = ' '.join(data_lemmatize)
  return data_fixed

In [ ]:
question_clean_train = [clean_data_question(sentence) for sentence in question_train['question']]
question_clean_test = [clean_data_question(sentence) for sentence in question_test['question']]

In [ ]:
question_train['cleaned_data'] = question_clean_train
question_test['cleaned_data'] = question_clean_test

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### Word2Vec

In [ ]:
import numpy as np
aa=np.concatenate((docs_clean_train,question_clean_train))

In [ ]:
from gensim.models import Word2Vec

# Training a word2vec model from the given data set
w2v_model = Word2Vec(aa, size=300, min_count=2,window=5, sg=1,workers=4)

### Get Embedding

In [ ]:
# Function returning vector reperesentation of a document
def get_embedding_w2v(doc_tokens):
    embeddings = []
    if len(doc_tokens)<1:
        return np.zeros(300)
    else:
        for tok in doc_tokens:
            if tok in w2v_model.wv.vocab:
                embeddings.append(w2v_model.wv.word_vec(tok))
            else:
                embeddings.append(np.random.rand(300))
        # mean the vectors of individual words to get the vector of the document
        return np.mean(embeddings, axis=0)

# Getting Word2Vec Vectors for Testing Corpus and Queries
docs_test['vector']=docs_test['cleaned_data'].apply(lambda x :get_embedding_w2v(x.split()))
question_test['vector']=question_test['cleaned_data'].apply(lambda x :get_embedding_w2v(x.split()))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## Similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def ranking_ir(query):
  
  # pre-process Query
  query=query.lower()
  query=word_tokenize(query)
  query=[lem.lemmatize(word,"v") for word in query]
  query=' '.join(query)

  # generating vector
  vector=get_embedding_w2v(query.split())

  # ranking documents
  documents=docs_test.copy()
  documents['similarity']=documents['vector'].apply(lambda x: cosine_similarity(np.array(vector).reshape(1, -1),np.array(x).reshape(1, -1)).item())
  documents.sort_values(by='similarity',ascending=False,inplace=True)
  
  return documents[['document_id','context','similarity']].tail(10).reset_index(drop=True)

In [ ]:
ranking_ir('pandemic COVID')

,document_id,context,similarity
0,2519,Detectable 2019-nCoV viral RNA in blood is a s...,0.919931
1,1559,COVID-19 and smoking: A systematic review of t...,0.919809
2,2668,2019-nCoV: The Identify-Isolate-Inform (3I) To...,0.919741
3,1545,Species‐specific clinical characteristics of h...,0.919715
4,2669,Frontiers in antiviral therapy and immunothera...,0.919696
5,2628,Haunted with and hunting for viruses\n\nhttps:...,0.919548
6,1552,One step closer to an experimental infection s...,0.919544
7,185,"CDC Summary 21 MAR 2020,\nhttps://www.cdc.gov/...",0.919402
8,1588,Prediction of lncRNA-protein interactions usin...,0.919323
9,1546,First Complete Genome Sequence of a French Bov...,0.918663
